# Выгрузка роликов по товарной категории
Пример выгрузки рекламных роликов по товарной категории

## Описание задачи и условий расчета
- Период: 01.02.2022-03.02.2022
- ЦА: Все 4+
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index
- Ролики: Товарная категория 2 - УСЛУГИ ФИНАНСОВЫЕ; Ролик распространение - Сетевой; Статус события - Реальный
- Статистики: Rtg%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники
Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# Найдем id товарная категория 2-го уровня - "УСЛУГИ ФИНАНСОВЫЕ"
# Для этого в параметр level передадим 2, в параметр name - название категории
cats.get_tv_article(levels=[2], name=['УСЛУГИ ФИНАНСОВЫЕ'])

In [ ]:
# Обратимся к словарю типов распространения
cats.get_tv_breaks_distribution()

# Для типа распространения "Сетевой" идентификатор - N

In [ ]:
# Обратимся к словарю статусов события
cats.get_tv_issue_status()

# Для "Реальный", будем использовать id R

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2022-02-01', '2022-02-03')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = None

# Фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов: Товарная категория 2 - УСЛУГИ ФИНАНСОВЫЕ; Ролик распространение - Сетевой; Статус события - Реальный
ad_filter = 'articleLevel2Id = 2272 and adDistributionType = N and adIssueStatusId = R'

# Указываем список срезов
slices = ['adSpotId', #Ролик ID выхода, обязательный атрибут! 
          'researchDate', #Дата, обязательный атрибут!
          'advertiserName', #Рекламодатель
          'brandName', #Бренд
          'adStartTime', #Ролик время начала
          'adFinishTime', #Ролик время окончания
          'tvCompanyName', #Телекомпания
          'adName', #Ролик
          'adTypeName' #Ролик тип
         ]
# Указываем список статистик для расчета
statistics = ['RtgPer']

# Задаем условия сортировки: Ролик ID выхода (по возр.)
sortings = {'adSpotId':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1 #TV Index Russia all
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_simple_task(task_json))

## Получение результата

В случае многотоварной рекламы у одного ролика может быть несколько продуктов, саббрендов, брендов и т.д. По умолчанию каждое значение выгружается отдельной строкой, поэтому в результате расчета возможны дубли по полю **adSpotId**

Для объединения атрибутов в списки необходимо в настройках метода `result2table` указать `to_lists=True`. В этом случае в поле **adSpotId** будут только уникальные значения. 

In [ ]:
# Получаем результат 
df = mtask.result2table(mtask.get_result(task_timeband), to_lists=True)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]

df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('02_article_level2_filter')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)